In [7]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import PowerTransformer, StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import Ridge, ElasticNet, LinearRegression
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

# 1. Feature engineering
def create_initial_features(df):
    df = df.copy()
    df['inv_prod_ratio'] = df['Current_Inventory(liters/kg)'] / (df['Produced_Quantity(liters/kg)']+1e-6)
    df['cost_sell_ratio'] = df['Unit_Cost_Price'] / (df['Unit_Selling_Price']+1e-6)
    df['profit_margin'] = df['Unit_Selling_Price'] - df['Unit_Cost_Price']
    df['prod_per_acre'] = df['Produced_Quantity(liters/kg)'] / (df['Land Size (acres)']+1e-6)
    df['inv_turnover'] = df['Produced_Quantity(liters/kg)'] / (df['Current_Inventory(liters/kg)']+1e-6)
    df['storage_cost'] = df['Storage_Duration(days)'] * df['Unit_Cost_Price']
    df['prod_storage'] = df['Produced_Quantity(liters/kg)']*df['Storage_Duration(days)']
    df['land_prod'] = df['Land Size (acres)']*df['Produced_Quantity(liters/kg)']
    # log transforms
    for c in ['Land Size (acres)','Produced_Quantity(liters/kg)',
              'Current_Inventory(liters/kg)','Storage_Duration(days)']:
        df[f'log_{c}'] = np.log1p(df[c])
    return df

# 2. Outlier flag
def flag_outliers(y):
    low, high = y.quantile(0.01), y.quantile(0.99)
    return ((y<low)|(y>high)).astype(int)

# 3. RMSE
def rmse(a,b): return np.sqrt(mean_squared_error(a,b))

# 4. Bayesian optimize RF
def optimize_rf(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 500),
        'max_depth': trial.suggest_int('max_depth', 8, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 4),
        'max_features': trial.suggest_categorical('max_features',['sqrt','log2',None]),
        'random_state':42, 'n_jobs':-1
    }
    kf = KFold(3, shuffle=True, random_state=42)
    vals=[]
    for tr, val in kf.split(X):
        m=RandomForestRegressor(**params).fit(X.iloc[tr],y.iloc[tr])
        vals.append(rmse(y.iloc[val],m.predict(X.iloc[val])))
    return np.mean(vals)

# 5. OOF generator
def get_oof(model, X, y, X_test, folds=10):
    oof = np.zeros(len(X))
    oof_test=np.zeros(len(X_test))
    test_folds=np.zeros((folds,len(X_test)))
    kf=KFold(folds,shuffle=True,random_state=42)
    for i,(tr,va) in enumerate(kf.split(X)):
        Xtr,ytr=X.iloc[tr],y.iloc[tr]
        Xva,yva=X.iloc[va],y.iloc[va]
        model.fit(Xtr,ytr)
        oof[va]=model.predict(Xva)
        test_folds[i]=model.predict(X_test)
    oof_test[:] = test_folds.mean(0)
    return oof.reshape(-1,1), oof_test.reshape(-1,1)

def main():
    # Load
    tr=pd.read_csv('Train.csv'); te=pd.read_csv('Test.csv'); sub=pd.read_csv('Submission.csv')
    trf=create_initial_features(tr)
    tef=create_initial_features(te)
    TARGET=trf['Reorder_Point(liters/kg)']
    
    # Outlier flag
    out_flag=flag_outliers(TARGET)
    
    # Split inliers and outliers
    X_all=trf.drop('Reorder_Point(liters/kg)',axis=1)
    X_train_in=X_all[out_flag==0]; y_in=TARGET[out_flag==0]
    X_train_out=X_all[out_flag==1]; y_out=TARGET[out_flag==1]
    
    X_test=tef[X_train_in.columns]
    
    # Feature selection: initial RF on inliers
    rf0=RandomForestRegressor(n_estimators=200,max_depth=12,random_state=42,n_jobs=-1)
    rf0.fit(X_train_in,y_in)
    imp_df=pd.Series(rf0.feature_importances_,index=X_train_in.columns).sort_values(ascending=False)
    top_feats=imp_df.iloc[:25].index.tolist()
    
    # Interaction expansion on top features
    poly=PolynomialFeatures(degree=2,interaction_only=True,include_bias=False)
    Xi_in=poly.fit_transform(X_train_in[top_feats])
    Xi_out=poly.transform(X_train_out[top_feats])
    Xi_test=poly.transform(X_test[top_feats])
    
    # Scaling
    pt=PowerTransformer(method='yeo-johnson')
    Xi_in=pt.fit_transform(Xi_in)
    Xi_out=pt.transform(Xi_out)
    Xi_test=pt.transform(Xi_test)
    
    # Bayesian optimize RF on inliers
    study=optuna.create_study(direction='minimize')
    study.optimize(lambda t: optimize_rf(t, pd.DataFrame(Xi_in), pd.Series(y_in)),n_trials=25)
    rf_best=RandomForestRegressor(**study.best_params,random_state=42,n_jobs=-1)
    
    # Other base models
    et=ExtraTreesRegressor(n_estimators=300,random_state=42,n_jobs=-1)
    lgbm=lgb.LGBMRegressor(n_estimators=500,learning_rate=0.05,max_depth=8,random_state=42)
    cat=CatBoostRegressor(iterations=500,learning_rate=0.05,depth=6,verbose=0,random_seed=42)
    
    # OOF
    rf_o, rf_t = get_oof(rf_best, pd.DataFrame(Xi_in), pd.Series(y_in), Xi_test)
    et_o, et_t = get_oof(et, pd.DataFrame(Xi_in), pd.Series(y_in), Xi_test)
    l_o,  l_t  = get_oof(lgbm, pd.DataFrame(Xi_in), pd.Series(y_in), Xi_test)
    c_o,  c_t  = get_oof(cat, pd.DataFrame(Xi_in), pd.Series(y_in), Xi_test)
    
    # Stack layer1
    Xs=np.hstack([rf_o,et_o,l_o,c_o])
    Xt=np.hstack([rf_t,et_t,l_t,c_t])
    sc=StandardScaler()
    Xs_s=sc.fit_transform(Xs); Xt_s=sc.transform(Xt)
    
    # Layer2 meta
    rmeta=Ridge(alpha=3,random_state=42)
    emeta=ElasticNet(alpha=0.01,l1_ratio=0.7,random_state=42)
    rmeta.fit(Xs_s,y_in); emeta.fit(Xs_s,y_in)
    p_r=rmeta.predict(Xt_s); p_e=emeta.predict(Xt_s)
    
    # Combine inliers+outliers predictions
    final = p_r*0.6 + p_e*0.4  # weighted meta blend
    # For outliers, simple RF on Xi_out
    rf_out=RandomForestRegressor(n_estimators=200,max_depth=12,random_state=42,n_jobs=-1)
    rf_out.fit(Xi_out,y_out)
    out_preds=rf_out.predict(Xi_test)
    # blend by out_flag probability (use baseline)
    blend = np.mean(out_flag)  # approx 2% outliers
    final = final*(1-blend) + out_preds*blend
    
    # Clip and save
    final = np.clip(final,TARGET.quantile(0.01),TARGET.quantile(0.99))
    sub['Reorder_Point(liters/kg)']=final
    sub.to_csv('advanced_submission.csv',index=False)
    print("OOF Ridge RMSE:", rmse(y_in, rmeta.predict(Xs_s)))
    print("Saved advanced_submission.csv")

if __name__=='__main__':
    main()


[I 2025-08-04 18:46:02,640] A new study created in memory with name: no-name-ee9ce872-a492-42e6-b77e-ac847280f3c1
[I 2025-08-04 18:46:07,054] Trial 0 finished with value: 20.56300789002034 and parameters: {'n_estimators': 340, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'log2'}. Best is trial 0 with value: 20.56300789002034.
[I 2025-08-04 18:48:12,193] Trial 1 finished with value: 20.65226901165148 and parameters: {'n_estimators': 387, 'max_depth': 19, 'min_samples_split': 9, 'min_samples_leaf': 3, 'max_features': None}. Best is trial 0 with value: 20.56300789002034.
[I 2025-08-04 18:49:17,104] Trial 2 finished with value: 20.541647419772666 and parameters: {'n_estimators': 228, 'max_depth': 13, 'min_samples_split': 7, 'min_samples_leaf': 1, 'max_features': None}. Best is trial 2 with value: 20.541647419772666.
[I 2025-08-04 18:49:22,499] Trial 3 finished with value: 20.560649925836017 and parameters: {'n_estimators': 363, 'max_depth': 15, 'min_sampl

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003348 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 43605
[LightGBM] [Info] Number of data points in the train set: 8435, number of used features: 171
[LightGBM] [Info] Start training from score 54.988321
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai